In [9]:
cd manifests

/home/ubuntu/mlops_workshops/seldon/manifests


python component details:
https://docs.seldon.io/projects/seldon-core/en/latest/python/python_component.html

model,combiner,router:
https://docs.seldon.io/projects/seldon-core/en/latest/examples/graph-metadata.html#Two-Level-Graph

### Let's deploy a Housing Preprocessor that gets the data from the feature store and standardize it

In [ ]:
!kubectl apply -f seldondeployment1.yaml

In [ ]:
# in terminal: kubectl port-forward svc/seldon-preprocessor-model-model 9100:9000

In [138]:
import requests
response = requests.post('http://localhost:9100/api/v1.0/predictions', json={"data":{"ndarray":[[1], [5]]}}).json()
response

{'data': {'names': ['house_main_view:HouseId',
   'house_main_view:MedInc',
   'house_main_view:HouseAge',
   'house_main_view:AveRooms',
   'house_main_view:AveBedrms',
   'house_main_view:Population',
   'house_main_view:AveOccup',
   'house_lat_lon_view:Latitude',
   'house_lat_lon_view:Longitude'],
  'ndarray': [[-1.7319668924113674,
    -0.7324396877792114,
    -0.3686446776412593,
    -0.7979812453599388,
    0.07906991444149086,
    0.7289808981819601,
    0.08193376858427932,
    -0.6797418814172994,
    0.5938194097272528],
   [-1.7312955548882658,
    -1.3334053246300217,
    -0.8453931491070594,
    -0.34815492500568834,
    -0.019726266386578314,
    1.1316531272558914,
    -0.018082498363888314,
    1.6377819950507821,
    -0.9834354838678211]]},
 'meta': {'requestPath': {'model': 'dvoitekh/seldon_feast:workshop13'}}}

In [32]:
response['data']['ndarray']

[[-1.7319668924113674,
  -0.7324396877792114,
  -0.3686446776412593,
  -0.7979812453599388,
  0.07906991444149086,
  0.7289808981819601,
  0.08193376858427932,
  -0.6797418814172994,
  0.5938194097272528],
 [-1.7312955548882658,
  -1.3334053246300217,
  -0.8453931491070594,
  -0.34815492500568834,
  -0.019726266386578314,
  1.1316531272558914,
  -0.018082498363888314,
  1.6377819950507821,
  -0.9834354838678211]]

In [33]:
!kubectl delete -f seldondeployment1.yaml

seldondeployment.machinelearning.seldon.io "seldon-preprocessor" deleted


### Let's deploy a model that consumes this data

In [10]:
!kubectl apply -f seldondeployment2.yaml

seldondeployment.machinelearning.seldon.io/seldon-regressor created


In [ ]:
# in terminal: kubectl port-forward svc/seldon-regressor-model-model 9200:9000

In [150]:
response = requests.post('http://localhost:9200/api/v1.0/predictions', json={"data":{"ndarray": [[-1.7312955548882658,
  -1.3334053246300217,
  -0.8453931491070594,
  -0.34815492500568834,
  -0.019726266386578314,
  1.1316531272558914,
  -0.018082498363888314,
  1.6377819950507821,
  -0.9834354838678211]]}}).json()
response

{'data': {'names': [], 'ndarray': [0.596]},
 'meta': {'requestPath': {'model': 'dvoitekh/seldon_feast:workshop13'}}}

In [ ]:
!kubectl delete -f seldondeployment2.yaml

### Let's deploy them together as a pipeline!

In [ ]:
!kubectl apply -f seldondeployment3.yaml

In [ ]:
!kubectl get po

In [ ]:
!kubectl get svc

In [ ]:
# in terminal: kubectl port-forward svc/seldon-pipeline-model 8100:8000

In [184]:
response = requests.post('http://localhost:8100/api/v1.0/predictions', json={"data":{"ndarray":[[2], [5]]}}).json()
response

{'data': {'names': [], 'ndarray': [2.017, 0.596]},
 'meta': {'requestPath': {'preprocessor': 'dvoitekh/seldon_feast:workshop13',
   'regressor': 'dvoitekh/seldon_feast:workshop13'}}}

In [ ]:
!kubectl delete -f seldondeployment3.yaml

### Let's add outliers detector with Combiner

In [ ]:
!kubectl apply -f seldondeployment4.yaml

In [ ]:
!kubectl get po

In [ ]:
# in terminal: kubectl port-forward svc/seldon-combiner-combiner 8100:8000

In [48]:
response = requests.post('http://localhost:8100/api/v1.0/predictions', json={"data":{"ndarray":[[154], [5]]}}).json()
response

{'jsonData': {'outliers_detector': [1, 0], 'regressor': [0.875, 0.596]},
 'meta': {'requestPath': {'combiner': 'dvoitekh/seldon_feast:workshop13',
   'outlier-detector': 'dvoitekh/seldon_feast:workshop13',
   'preprocessor': 'dvoitekh/seldon_feast:workshop13',
   'regressor': 'dvoitekh/seldon_feast:workshop13'}}}

In [50]:
!kubectl delete -f seldondeployment4.yaml

### Let's create default A/B test (Only 2 hands are supported)

In [69]:
!kubectl apply -f seldondeployment5.0.yaml

seldondeployment.machinelearning.seldon.io/seldon-ab-test created


In [88]:
!kubectl get po

NAME                                                              READY   STATUS    RESTARTS   AGE
seldon-ab-test-ab-test-0-preprocessor-model1-model2-56bd55w8hx8   4/4     Running   0          25s


In [ ]:
# in terminal: kubectl port-forward svc/seldon-ab-test-ab-test 8100:8000

In [96]:
response = requests.post('http://localhost:8100/api/v1.0/predictions', json={"data":{"ndarray":[[154], [5]]}}).json()
response

{'data': {'names': [], 'ndarray': [0.875, 0.596]},
 'meta': {'requestPath': {'model1': 'dvoitekh/seldon_feast:workshop13',
   'preprocessor': 'dvoitekh/seldon_feast:workshop13'}}}

In [ ]:
!kubectl delete -f seldondeployment5.0.yaml

### Let's create a Random Router

Available routers: https://docs.seldon.io/projects/seldon-core/en/latest/analytics/routers.html

Example of Eps-greedy: https://github.com/SeldonIO/seldon-core/blob/master/components/routers/epsilon-greedy/README.md

In [53]:
!kubectl apply -f seldondeployment5.yaml

In [ ]:
!kubectl get po

In [ ]:
# in terminal: kubectl port-forward svc/seldon-router-router 8100:8000

In [64]:
response = requests.post('http://localhost:8100/api/v1.0/predictions', json={"data":{"ndarray":[[154], [5]]}}).json()
response

{'data': {'names': [], 'ndarray': [0.875, 0.596]},
 'meta': {'requestPath': {'model1': 'dvoitekh/seldon_feast:workshop13',
   'preprocessor': 'dvoitekh/seldon_feast:workshop13'},
  'routing': {'model1': -1, 'preprocessor': -1, 'router': 0}}}

In [ ]:
response = requests.post('http://localhost:8100/api/v1.0/feedback', json={
    "request": {"ndarray":[[154], [5]]},
    "response": response,
    "reward": -1,
    "truth": {"data": {"ndarray": [1, 1]}}
}).json()
response

In [ ]:
!kubectl logs -f seldon-router-router-0-preprocessor-model1-model2-router-5cvpwz router

In [66]:
!kubectl delete -f seldondeployment5.yaml

seldondeployment.machinelearning.seldon.io "seldon-router" deleted


### Also, it's possible to perform streaming via TFServing: https://docs.seldon.io/projects/seldon-core/en/latest/examples/cifar10_kafka.html